May: **Research Questions 2.1** (Ideal and Real)

# Which of the three upwelling mechanisms (time dependence, advection, convergence of isobaths, coastal trapped wave) provides the strongest contribution to upwelling in Mackenzie Canyon?

Where does upwelling occur?

In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/upwelling_depth/')
import upwelling_functions

import warnings
warnings.filterwarnings("ignore")

# Functions

In [2]:
def plot_original_depth(depth_upwelled, tmask, deptht, dep_ind_slice, levels, kind, case):
    cmap, norm = cmo.cm.delta, upwelling_functions.MidpointNormalize(midpoint=0)
    cmap.set_bad('silver')

    vmin, vmax = int(np.min(depth_upwelled)), int(np.max(depth_upwelled))
    xs = np.arange(tmask.shape[-1])
    ys = np.arange(tmask.shape[-2])

    fig, axes = plt.subplots(3, 3, figsize = (20, 21))
    for ax, n in zip(axes.flatten(), np.arange(9)):
        p = ax.pcolormesh(xs, ys, depth_upwelled[n, :, :], cmap=cmap, vmin=vmin, vmax=vmax, norm=norm)
        cs = ax.contour(xs, ys, depth_upwelled[n, :, :], levels = levels, colors='k', alpha=0.8)
        ax.clabel(cs, inline=1, fontsize=10, fmt ='%r', colors='k')
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect(aspect='equal')
        ax.set_xlim([0, xs[-1]])
        ax.set_ylim([0, ys[-1]])
        ax.set_title('Day ' + str(n+1), fontsize=20)
        cbar = fig.colorbar(p, ax=ax, fraction=0.05, orientation='horizontal', pad=0.009)
        cbar.ax.tick_params(labelsize=13)
        tick_locator = ticker.MaxNLocator(nbins=15)
        cbar.locator = tick_locator
        cbar.update_ticks()

    fig.tight_layout(w_pad=1.2, h_pad=0.01)
    fig.suptitle(kind + ' - ' + case +': ' + 'displacement depth [m] at depth = '+
                 str(int(deptht[dep_ind_slice]))+' m', fontsize=25)
    plt.subplots_adjust(top=0.96)

    return fig

# Constants

In [3]:
vosaline_ref = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/conditions/NEMO_files/salinity/salinity_for_agrif.nc')['vosaline'][:]
vosaline_ref_profile = vosaline_ref[0, :, 0, 0]

fname = '1_MCKNZ_1h_20170101_201701*'
meshmaskname = '1_mesh_mask.nc'

days = 9 
time_ind = 24*days
levels = [20, 50, 80, 100, 120]
dep_ind_slices = [1, 9, 16]

# Ideal

In [4]:
kind = 'ideal'
case = 'base'

dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

In [5]:
for dep_ind_slice, n in zip(dep_ind_slices, np.arange(len(dep_ind_slices))):
    vosaline, sozotaux, deptht, tmask = upwelling_functions.get_vars_salt(dirname, fname, meshmaskname, dep_ind_slice, time_ind, None)

    vosaline_daily = upwelling_functions.get_daily_vosaline(vosaline, tmask, days)

    depth_ind_daily = upwelling_functions.get_daily_depth_ind(vosaline_daily, vosaline_ref_profile)
    depth_m_daily, depth_upwelled = upwelling_functions.get_daily_depth_m(deptht, dep_ind_slice, depth_ind_daily, tmask)
    
    fig = plot_original_depth(depth_upwelled, tmask, deptht, dep_ind_slice, levels, kind, case)
    fig.savefig('../writing_images/displaced_dep_'+kind+'_'+case+'_'+str(dep_ind_slice)+'_daily.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 

0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8


# Real

In [4]:
kind = 'real'
case = 'base'

dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'

In [5]:
for dep_ind_slice, n in zip(dep_ind_slices, np.arange(len(dep_ind_slices))):
    vosaline, sozotaux, deptht, tmask = upwelling_functions.get_vars_salt(dirname, fname, meshmaskname, dep_ind_slice, time_ind, None)

    vosaline_daily = upwelling_functions.get_daily_vosaline(vosaline, tmask, days)

    depth_ind_daily = upwelling_functions.get_daily_depth_ind(vosaline_daily, vosaline_ref_profile)
    depth_m_daily, depth_upwelled = upwelling_functions.get_daily_depth_m(deptht, dep_ind_slice, depth_ind_daily, tmask)
    
    fig = plot_original_depth(depth_upwelled, tmask, deptht, dep_ind_slice, levels, kind, case)
    fig.savefig('../writing_images/displaced_dep_'+kind+'_'+case+'_'+str(dep_ind_slice)+'_daily.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 

0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
